In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, model_selection
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict
import math

In [2]:
df = pd.read_csv('GOOG.csv', delimiter=',')
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-08-05,1170.040039,1175.239990,1140.140015,1152.319946,1152.319946,2597500
1,2019-08-06,1163.310059,1179.959961,1160.000000,1169.949951,1169.949951,1709400
2,2019-08-07,1156.000000,1178.444946,1149.624023,1173.989990,1173.989990,1444300
3,2019-08-08,1182.829956,1205.010010,1173.020020,1204.800049,1204.800049,1468000
4,2019-08-09,1197.989990,1203.880005,1183.603027,1188.010010,1188.010010,1065700
5,2019-08-12,1179.209961,1184.959961,1167.671997,1174.709961,1174.709961,1003000
6,2019-08-13,1171.459961,1204.780029,1171.459961,1197.270020,1197.270020,1294400
7,2019-08-14,1176.310059,1182.300049,1160.540039,1164.290039,1164.290039,1578700
8,2019-08-15,1163.500000,1175.839966,1162.109985,1167.260010,1167.260010,1218700
9,2019-08-16,1179.550049,1182.719971,1171.810059,1177.599976,1177.599976,1313300


In [3]:
# Close and Adj Close hold the same values
for item1, item2 in zip(df['Close'], df['Adj Close']):
    result = (item1 == item2)
print(result)

True


In [4]:
df['Pct_Change'] = (df['Close'] - df['Open']) / df['Open']*100
df.set_index('Date', drop=True, inplace=True)

In [5]:
df.head(5)

,Open,High,Low,Close,Adj Close,Volume,Pct_Change
Date,,,,,,,
2019-08-05,1170.040039,1175.239990,1140.140015,1152.319946,1152.319946,2597500,-1.514486
2019-08-06,1163.310059,1179.959961,1160.000000,1169.949951,1169.949951,1709400,0.570776
2019-08-07,1156.000000,1178.444946,1149.624023,1173.989990,1173.989990,1444300,1.556228
2019-08-08,1182.829956,1205.010010,1173.020020,1204.800049,1204.800049,1468000,1.857418
2019-08-09,1197.989990,1203.880005,1183.603027,1188.010010,1188.010010,1065700,-0.833060


In [6]:
df.isnull().sum()

Open          0
High          0
Low           0
Close         0
Adj Close     0
Volume        0
Pct_Change    0
dtype: int64

In [7]:
df = df[['Close', 'Pct_Change', 'Volume']]

In [8]:
# Forecast 1% of the date in the future
forecast_col = 'Close'
forecast_out = math.ceil(0.01*len(df))
df['Label'] = df[forecast_col].shift(-forecast_out)
print(df)

                  Close  Pct_Change   Volume        Label
Date                                                     
2019-08-05  1152.319946   -1.514486  2597500  1204.800049
2019-08-06  1169.949951    0.570776  1709400  1188.010010
2019-08-07  1173.989990    1.556228  1444300  1174.709961
2019-08-08  1204.800049    1.857418  1468000  1197.270020
2019-08-09  1188.010010   -0.833060  1065700  1164.290039
...                 ...         ...      ...          ...
2020-07-28  1500.339966   -1.628666  1702200  1482.959961
2020-07-29  1522.020020    1.042280  1106500  1474.449951
2020-07-30  1531.449951    2.301266  1671400          NaN
2020-07-31  1482.959961   -1.465110  3439900          NaN
2020-08-03  1474.449951   -0.819974  2330200          NaN

[252 rows x 4 columns]


In [12]:
# x is features to predict y, y is the price prediction
df.dropna(inplace=True)
x = np.array(df.drop(['Label'], 1))
x = preprocessing.scale(x)
y = np.array(df['Label'])
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.3)
linear = LinearRegression()
linear.fit(x,y)
print(f"Accuracy of the model is {linear.score(x_test, y_test)}")

Accuracy of the model is 0.9137561761002061
